In [1]:
import sys
sys.executable

'C:\\Users\\vicso\\anaconda3\\envs\\myenv\\python.exe'

In [2]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   - -------------------------------------- 3.4/111.0 MB 18.3 MB/s eta 0:00:06
   -- ------------------------------------- 7.1/111.0 MB 17.4 MB/s eta 0:00:06
   --- ------------------------------------ 11.0/111.0 MB 17.6 MB/s eta 0:00:06
   ----- ---------------------------------- 14.7/111.0 MB 17.8 MB/s eta 0:00:06
   ------ --------------------------------- 18.1/111.0 MB 17.3 MB/s eta 0:00:06
   ------- -------------------------------- 22.0/111.0 MB 17.4 MB/s eta 0:00:06
   --------- ------------------------------ 25.7/111.0 MB 17.3 MB/s eta 0:00:05
   ---------- ----------------------------- 29.1/111.0 MB 17.4 MB/s eta 0:00:05
   ----------- ---------------------------- 32.8/111.0 MB 17.3 MB/s eta 0:00:05
   ------------- -------------------------- 36.4/111.0 MB 17.4 MB/s eta 0:00:05
   -------------- ------------------------- 40.4/111.0 MB 17.3 MB/s eta 0:00:05
   --------------- ------------------------ 44.0/11

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!{sys.executable} -m pip install matplotlib scikit-learn pillow tqdm

In [1]:
import torch, torchvision
import matplotlib
import sklearn
from PIL import Image
from tqdm import tqdm

print("All imports successful ✅")


All imports successful ✅


In [2]:
import os, glob
from collections import Counter
#tells me whats inside my dataset


DATA_DIR = r"C:\Users\vicso\Downloads\ThaiBankNotes\ThaiBankNotes\Training"

classes = sorted([
    d for d in os.listdir(DATA_DIR)
    if os.path.isdir(os.path.join(DATA_DIR, d))
])

counts = {}
for c in classes:
    folder = os.path.join(DATA_DIR, c)
    imgs = (
        glob.glob(os.path.join(folder, "*.jpg")) +
        glob.glob(os.path.join(folder, "*.JPG")) +
        glob.glob(os.path.join(folder, "*.png"))
    )
    counts[c] = len(imgs)

print("Classes:")
for k, v in counts.items():
    print(f"{k}: {v}")

print("\nTotal images:", sum(counts.values()))


Classes:
THAI100: 320
THAI1000: 320
THAI20: 320
THAI50: 320
THAI500: 320

Total images: 1600


In [3]:
from pathlib import Path

BASE_DIR = Path(r"C:\Users\vicso\Downloads\ThaiBankNotes\ThaiBankNotes")
TRAIN_DIR = BASE_DIR / "Training"
VAL_DIR   = BASE_DIR / "Validation"


In [4]:
import os
from torchvision.datasets import ImageFolder
from PIL import Image

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG"}

def is_valid_file(path: str) -> bool:
    return os.path.splitext(path)[1] in IMG_EXTS

# quick check: how many non-images are present?
def count_non_images(root):
    n = 0
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if os.path.splitext(f)[1] not in IMG_EXTS:
                n += 1
    return n

print("Non-images in Training:", count_non_images(TRAIN_DIR))
print("Non-images in Validation:", count_non_images(VAL_DIR))


Non-images in Training: 800
Non-images in Validation: 200


In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_ds = ImageFolder(root=str(TRAIN_DIR), transform=train_tfms, is_valid_file=is_valid_file)
val_ds   = ImageFolder(root=str(VAL_DIR), transform=val_tfms,   is_valid_file=is_valid_file)

print("Classes:", train_ds.classes)
print("Train size:", len(train_ds))
print("Val size:", len(val_ds))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)


Classes: ['THAI100', 'THAI1000', 'THAI20', 'THAI50', 'THAI500']
Train size: 800
Val size: 200


In [20]:
import numpy as np
import torch
from torch.utils.data import Subset, DataLoader

SEED = 42
TEST_FRAC = 0.15

targets = np.array(train_ds.targets)  # class index for each sample
num_classes = len(train_ds.classes)

rng = np.random.default_rng(SEED)

train_indices = []
test_indices = []

for c in range(num_classes):
    idx_c = np.where(targets == c)[0]
    rng.shuffle(idx_c)

    n_test = int(round(len(idx_c) * TEST_FRAC))
    test_indices.extend(idx_c[:n_test].tolist())
    train_indices.extend(idx_c[n_test:].tolist())

# shuffle overall indices (optional)
rng.shuffle(train_indices)
rng.shuffle(test_indices)

train_split = Subset(train_ds, train_indices)
test_split  = Subset(train_ds, test_indices)

print("Train split size:", len(train_split))
print("Test split size:", len(test_split))
print("Val size (unchanged):", len(val_ds))


Train split size: 680
Test split size: 120
Val size (unchanged): 200


In [21]:
train_loader = DataLoader(train_split, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,      batch_size=32, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_split,  batch_size=32, shuffle=False, num_workers=0)

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        preds = model(x).argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_true.extend(y.numpy())

cm = confusion_matrix(all_true, all_preds)
print("Confusion matrix (TEST):\n", cm)
print("\nReport (TEST):\n", classification_report(all_true, all_preds, target_names=train_ds.classes))


NameError: name 'model' is not defined

In [9]:
import torch
from torch import nn
from torchvision import models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(train_ds.classes))
model = model.to(DEVICE)


Device: cpu
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\vicso/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:06<00:00, 17.0MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [11]:
from tqdm import tqdm

def run_epoch(model, loader, train=True):
    model.train() if train else model.eval()
    total_loss, total_correct, total = 0.0, 0, 0

    for x, y in tqdm(loader, leave=False):
        x, y = x.to(DEVICE), y.to(DEVICE)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            logits = model(x)
            loss = criterion(logits, y)
            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return total_loss / total, total_correct / total

for epoch in range(1, 6):
    train_loss, train_acc = run_epoch(model, train_loader, train=True)
    val_loss, val_acc = run_epoch(model, val_loader, train=False)
    print(f"Epoch {epoch}: train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")


Epoch 1: train_acc=0.468, val_acc=0.790


Epoch 2: train_acc=0.962, val_acc=0.940


Epoch 3: train_acc=0.993, val_acc=0.975


Epoch 4: train_acc=0.997, val_acc=0.970


Epoch 5: train_acc=0.994, val_acc=0.960


In [12]:
best_val_acc = -1.0
BEST_PATH = "resnet50_thai_rgb_best.pth"

for epoch in range(1, 6):
    train_loss, train_acc = run_epoch(model, train_loader, train=True)
    val_loss, val_acc = run_epoch(model, val_loader, train=False)
    print(f"Epoch {epoch}: train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), BEST_PATH)
        print(f"✅ Saved new best model (val_acc={best_val_acc:.3f})")


Epoch 1: train_acc=1.000, val_acc=0.985
✅ Saved new best model (val_acc=0.985)


Epoch 2: train_acc=0.997, val_acc=0.980


Epoch 3: train_acc=0.999, val_acc=0.970


Epoch 4: train_acc=0.991, val_acc=0.970


Epoch 5: train_acc=0.996, val_acc=0.980


In [13]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Load best checkpoint
BEST_PATH = "resnet50_thai_rgb_best.pth"
model.load_state_dict(torch.load(BEST_PATH, map_location=DEVICE))
model.eval()

all_preds, all_true = [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        preds = model(x).argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_true.extend(y.numpy())

cm = confusion_matrix(all_true, all_preds)
print("Confusion matrix (TEST):\n", cm)
print("\nReport (TEST):\n", classification_report(all_true, all_preds, target_names=train_ds.classes))


Confusion matrix (TEST):
 [[24  0  0  0  0]
 [ 0 24  0  0  0]
 [ 0  0 24  0  0]
 [ 0  0  0 24  0]
 [ 0  1  0  0 23]]

Report (TEST):
               precision    recall  f1-score   support

     THAI100       1.00      1.00      1.00        24
    THAI1000       0.96      1.00      0.98        24
      THAI20       1.00      1.00      1.00        24
      THAI50       1.00      1.00      1.00        24
     THAI500       1.00      0.96      0.98        24

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [16]:
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, Subset

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG"}

def is_valid_file(path: str) -> bool:
    return os.path.splitext(path)[1] in IMG_EXTS

# Grayscale transforms:
# - Grayscale(num_output_channels=3) keeps 3 channels so ResNet50 still works unchanged.
train_tfms_gray = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # keep ImageNet normalization
                         std=[0.229, 0.224, 0.225]),
])

val_tfms_gray = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Recreate ImageFolder datasets but with grayscale transforms
train_ds_gray = ImageFolder(root=str(TRAIN_DIR), transform=train_tfms_gray, is_valid_file=is_valid_file)
val_ds_gray   = ImageFolder(root=str(VAL_DIR),   transform=val_tfms_gray,   is_valid_file=is_valid_file)

# Sanity: classes should match exactly
print("Classes:", train_ds_gray.classes)


Classes: ['THAI100', 'THAI1000', 'THAI20', 'THAI50', 'THAI500']


In [23]:
train_split_gray = Subset(train_ds_gray, train_indices)
test_split_gray  = Subset(train_ds_gray, test_indices)

train_loader_gray = DataLoader(train_split_gray, batch_size=32, shuffle=True,  num_workers=0)
val_loader_gray   = DataLoader(val_ds_gray,      batch_size=32, shuffle=False, num_workers=0)
test_loader_gray  = DataLoader(test_split_gray,  batch_size=32, shuffle=False, num_workers=0)

print("Gray Train:", len(train_split_gray), "Gray Val:", len(val_ds_gray), "Gray Test:", len(test_split_gray))


Gray Train: 680 Gray Val: 200 Gray Test: 120


In [24]:
import torch
from torch import nn
from torchvision import models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

model_gray = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model_gray.fc = nn.Linear(model_gray.fc.in_features, len(train_ds_gray.classes))
model_gray = model_gray.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_gray.parameters(), lr=1e-4, weight_decay=1e-4)


Device: cpu


In [25]:
best_val_acc = -1.0
BEST_GRAY_PATH = "resnet50_thai_gray_best.pth"

for epoch in range(1, 6):
    train_loss, train_acc = run_epoch(model_gray, train_loader_gray, train=True)
    val_loss, val_acc     = run_epoch(model_gray, val_loader_gray,   train=False)
    print(f"[GRAY] Epoch {epoch}: train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model_gray.state_dict(), BEST_GRAY_PATH)
        print(f"✅ Saved new best GRAY model (val_acc={best_val_acc:.3f})")


[GRAY] Epoch 1: train_acc=0.394, val_acc=0.555
✅ Saved new best GRAY model (val_acc=0.555)


[GRAY] Epoch 2: train_acc=0.801, val_acc=0.765
✅ Saved new best GRAY model (val_acc=0.765)


[GRAY] Epoch 3: train_acc=0.935, val_acc=0.890
✅ Saved new best GRAY model (val_acc=0.890)


[GRAY] Epoch 4: train_acc=0.965, val_acc=0.945
✅ Saved new best GRAY model (val_acc=0.945)


[GRAY] Epoch 5: train_acc=0.979, val_acc=0.955
✅ Saved new best GRAY model (val_acc=0.955)


In [26]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model_gray.load_state_dict(torch.load(BEST_GRAY_PATH, map_location=DEVICE))
model_gray.eval()

all_preds, all_true = [], []
with torch.no_grad():
    for x, y in test_loader_gray:
        x = x.to(DEVICE)
        preds = model_gray(x).argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_true.extend(y.numpy())

cm = confusion_matrix(all_true, all_preds)
print("Confusion matrix (GRAY TEST):\n", cm)
print("\nReport (GRAY TEST):\n", classification_report(all_true, all_preds, target_names=train_ds_gray.classes))


Confusion matrix (GRAY TEST):
 [[24  0  0  0  0]
 [ 0 24  0  0  0]
 [ 0  0 24  0  0]
 [ 0  0  0 24  0]
 [ 0  1  0  0 23]]

Report (GRAY TEST):
               precision    recall  f1-score   support

     THAI100       1.00      1.00      1.00        24
    THAI1000       0.96      1.00      0.98        24
      THAI20       1.00      1.00      1.00        24
      THAI50       1.00      1.00      1.00        24
     THAI500       1.00      0.96      0.98        24

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120

